# Attachment Protocol Example - Alice

## Role: 

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


## Intialise a controller 

In [2]:
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

## Confirm that an active connection exists

This can fail sometimes, there are fallback steps described in Bob's notebook.

In [3]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':   
        global connection_id
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'initiator': 'external', 'created_at': '2020-11-03 18:00:38.610744Z', 'routing_state': 'none', 'invitation_mode': 'once', 'their_label': 'Bob', 'my_did': '2JSaZaqcASHPrVhfzLFrgm', 'connection_id': '6db10c92-61aa-4a2b-8e60-5a4cc02da454', 'state': 'request', 'accept': 'manual', 'request_id': '47a5a8fd-d26a-418f-b654-504a393ec461', 'invitation_key': '5mSAd7qvFnHVTiWsFtFXHNjSr4VokUka5MRZY7JcZ5Mz', 'updated_at': '2020-11-03 18:00:38.628262Z'}]
Connection : {'initiator': 'external', 'created_at': '2020-11-03 18:00:38.610744Z', 'routing_state': 'none', 'invitation_mode': 'once', 'their_label': 'Bob', 'my_did': '2JSaZaqcASHPrVhfzLFrgm', 'connection_id': '6db10c92-61aa-4a2b-8e60-5a4cc02da454', 'state': 'request', 'accept': 'manual', 'request_id': '47a5a8fd-d26a-418f-b654-504a393ec461', 'invitation_key': '5mSAd7qvFnHVTiWsFtFXHNjSr4VokUka5MRZY7JcZ5Mz', 'updated_at': '2020-11-03 18:00:38.628262Z'}
Connection is still progressing to active state, retry in a few moments


In [5]:
await attach_controller.connections.accept_request(results[0]["connection_id"])

ClientResponseError: 404, message='Record not found.', url=URL('http://alice-agent:8021/connections/6c932e68-907d-4158-92fc-4913bd1a2e67/accept-request')

Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/opt/conda/lib/python3.7/site-packages/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/opt/conda/lib/python3.7/site-packages/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.

### Note: We do not need to initialise any listeners for Alice. In this example she only sends attachments and does not receive them

## Make sure you have initialised the handler on [Bob's notebook](http://localhost:8889/notebooks/attachment.ipynb)

## Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [5]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': 'adc71765-2fa3-4027-af4a-e72797067c02'}


Here an image file is sent and is received by the handler in [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [15]:
data = FormData()
data.add_field('file',
               open('openmined.jpg', 'rb'),
               filename='openmined.jpg',
               content_type='image/jpeg')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': '4b541001-77ea-49b2-85ef-7c7bf9f06ceb'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [8]:
response = await attach_controller.terminate()
print(response)


None
